<a href="https://colab.research.google.com/github/bhadreshpsavani/ColabNotebooks/blob/master/DiffrentialPrivacyProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Diffrential Privacy Project

## MNIST Dataset Senario

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
!pip install syft
import matplotlib.pyplot as plt
import numpy as np
import torch

from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from sklearn.model_selection import train_test_split

     |████████████████████████████████| 215kB 2.7MB/s 
     |████████████████████████████████| 1.4MB 40.1MB/s 
     |████████████████████████████████| 450kB 38.9MB/s 
     |████████████████████████████████| 256kB 42.3MB/s 
     |████████████████████████████████| 81kB 22.4MB/s 
     |████████████████████████████████| 389kB 42.4MB/s 
     |████████████████████████████████| 204kB 40.5MB/s 
     |████████████████████████████████| 276kB 33.2MB/s 
     |████████████████████████████████| 51kB 19.1MB/s 
     |████████████████████████████████| 122kB 40.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [2]:
import torchvision.datasets as datasets
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

0it [00:00, ?it/s]

9920512it [00:02, 3530287.61it/s]                             


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 56139.60it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 936665.90it/s]                             
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 21113.13it/s]            

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [3]:
train_data = mnist_trainset.train_data
train_targets = mnist_trainset.train_labels

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [4]:
test_data = mnist_trainset.test_data
test_targets = mnist_trainset.test_labels

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [5]:
len(train_data)

60000

In [7]:
train_datasets=[]
train_lables=[]
j=0
for i in range(60):
  k=int((len(train_data)/60)*(i+1))
  train_datasets.append(train_data[j:k])
  train_lables.append(train_targets[j:k])
  j=k
k=0
j=0
for i in range(60):
  k=int((len(test_data)/60)*(i+1))
  train_datasets.append(test_data[j:k])
  train_lables.append(test_targets[j:k])
  j=k
print(len(train_datasets))

120


In [0]:
train_loaders=[]
test_loaders=[]
for a in range(119):
  
  X=train_datasets[a].numpy()
  y=train_lables[a].numpy()
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

  BATCH_SIZE = 32

  torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
  torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

  # create feature and targets tensor for test set.
  torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
  torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

  torch_X_train = torch_X_train.view(-1,1,28,28).float()
  torch_X_test = torch_X_test.view(-1,1,28,28).float()

  # Pytorch train and test sets
  train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
  test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

  # data loader
  train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
  test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)
  
  train_loaders.append(train_loader)
  test_loaders.append(test_loader)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.to(device);
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            var_X_batch = X_batch.float()
            var_y_batch = y_batch
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
    
            #print(correct)
       
        print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
            epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
    return model

In [0]:
def evaluate(model, test_loader):
#model = mlp
    correct = 0
    model.to(device)
    model.eval()
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs, test_labels=test_imgs.to(device),test_labels.to(device)
        test_imgs = test_imgs.float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))

In [13]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [15]:
for i in range(119):
  print("for model : "+str(i))
  cnn = CNN()
  fit(cnn, train_loaders[i])
  evaluate(cnn, test_loaders[i])
  model_name='drive/My Drive/Colab Notebooks/SecureAndPrivateAI/DiffrentialPrivacyProject/11Models/checkpoint_model_'+str(i)+'.pth'
  torch.save(cnn,model_name)
  
  print("model_"+str(i)+" is saved")

for model : 0
Epoch : 0 [468/850 (96%)]	Loss: 2.219300	 Accuracy:12.963%
Epoch : 1 [468/850 (96%)]	Loss: 2.035947	 Accuracy:19.676%
Epoch : 2 [468/850 (96%)]	Loss: 1.045288	 Accuracy:41.204%
Epoch : 3 [468/850 (96%)]	Loss: 0.290597	 Accuracy:62.731%
Epoch : 4 [468/850 (96%)]	Loss: 1.240679	 Accuracy:72.454%
Test accuracy:0.856% 


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


model_0 is saved
for model : 1
Epoch : 0 [468/850 (96%)]	Loss: 2.356295	 Accuracy:14.352%
Epoch : 1 [468/850 (96%)]	Loss: 1.717198	 Accuracy:22.685%
Epoch : 2 [468/850 (96%)]	Loss: 0.975337	 Accuracy:49.653%
Epoch : 3 [468/850 (96%)]	Loss: 0.732888	 Accuracy:67.708%
Epoch : 4 [468/850 (96%)]	Loss: 0.395243	 Accuracy:72.454%
Test accuracy:0.819% 
model_1 is saved
for model : 2
Epoch : 0 [468/850 (96%)]	Loss: 2.098316	 Accuracy:15.162%
Epoch : 1 [468/850 (96%)]	Loss: 1.266075	 Accuracy:38.657%
Epoch : 2 [468/850 (96%)]	Loss: 0.943176	 Accuracy:61.921%
Epoch : 3 [468/850 (96%)]	Loss: 0.740601	 Accuracy:71.644%
Epoch : 4 [468/850 (96%)]	Loss: 0.701883	 Accuracy:76.620%
Test accuracy:0.856% 
model_2 is saved
for model : 3
Epoch : 0 [468/850 (96%)]	Loss: 2.189446	 Accuracy:14.468%
Epoch : 1 [468/850 (96%)]	Loss: 1.457695	 Accuracy:27.315%
Epoch : 2 [468/850 (96%)]	Loss: 0.914384	 Accuracy:52.662%
Epoch : 3 [468/850 (96%)]	Loss: 0.470793	 Accuracy:66.898%
Epoch : 4 [468/850 (96%)]	Loss: 0.618

#Generate Labels for Target Dataset using 11 trained Models


In [0]:
model_list=[]
for i in range(119):
  model_list.append(torch.load('drive/My Drive/Colab Notebooks/SecureAndPrivateAI/DiffrentialPrivacyProject/11Models/checkpoint_model_'+str(i)+'.pth'))
  

In [17]:
print(len(model_list))

119


In [18]:
print(model_list[0])

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [0]:
def view_classify(img, ps, version="MNIST"):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    if version == "MNIST":
        ax2.set_yticklabels(np.arange(10))
    elif version == "Fashion":
        ax2.set_yticklabels(['T-shirt/top',
                            'Trouser',
                            'Pullover',
                            'Dress',
                            'Coat',
                            'Sandal',
                            'Shirt',
                            'Sneaker',
                            'Bag',
                            'Ankle Boot'], size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [0]:
model_labels_lists=[]
for model in model_list:
  labels_list=[]
  model.eval()
  for img in train_datasets[-1]:
    img=img.to(device)
    img=img.view(1,1,28,28).float()
    # Calculate the class probabilities (softmax) for img
    with torch.no_grad():
        output = model.forward(img)
#         ps=torch.exp(output)
#         view_classify(img.view(1,28,28).cpu(), ps.cpu(), version="MNIST")
    labels_list.append(torch.max(output,1)[1])
  model_labels_lists.append(labels_list)

In [0]:
label_list=[]
for i in range(1000):
  sub_list=[]
  for j in range(119):
    sub_list.append(model_labels_lists[j][i])
  label_list.append(sub_list)

In [24]:
print(len(label_list))
# test_labels=[]
# for i in test_targets[50000:60000]:
#   sub_list=[]
#   sub_list.append(i)
#   test_labels.append(sub_list)

119


#PATE Analysis

In [0]:
new_labels = list()
for an_image in label_list:

    label_counts = np.bincount(an_image, minlength=10)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [26]:
from syft.frameworks.torch.differential_privacy import pate

W0709 12:36:50.153824 139738690709376 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0709 12:36:50.171946 139738690709376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
preds=np.asarray(model_labels_lists).astype(int)
indices=np.asarray(new_labels).astype(int)

In [28]:
print(preds[:,:10], indices[:10])

[[6 3 5 ... 2 6 2]
 [6 2 5 ... 2 6 2]
 [6 3 5 ... 2 6 2]
 ...
 [6 3 5 ... 2 6 2]
 [6 2 5 ... 2 6 2]
 [6 2 5 ... 2 6 2]] [6 3 5 7 2 5 9 2 6 2]


In [29]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

In [30]:
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 51.51292546497024
Data Dependent Epsilon: 3.214316887866704


In [0]:
num_teachers, num_examples, num_labels = (10, 10000, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)


In [0]:
print(preds[:,:10], indices[:10])

[[9 4 5 0 1 9 2 4 8 6]
 [9 8 1 6 6 4 1 0 5 5]
 [1 2 2 5 1 4 4 4 8 9]
 [3 1 5 4 4 7 4 3 4 3]
 [0 0 9 6 9 6 6 3 0 3]
 [8 5 8 0 9 3 2 3 5 4]
 [1 2 8 7 8 3 5 9 3 6]
 [8 8 0 3 7 2 1 4 4 8]
 [3 8 2 6 4 2 2 7 7 0]
 [9 5 1 3 2 7 4 1 4 7]] [6 9 7 0 3 7 7 0 4 8]


In [0]:
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 411.5129254649703
Data Dependent Epsilon: 411.51292546502725


In [0]:
data_dep_eps, data_ind_eps=pate.perform_analysis_torch(preds, indices, 0.1, 1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

/usr/local/lib/python3.6/dist-packages/syft/frameworks/torch/differential_privacy/pate.py:353: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(counts, dtype=torch.float) - torch.tensor(counts[winner], dtype=torch.float)


Data Independent Epsilon: tensor(411.5129)
Data Dependent Epsilon: tensor(411.5247)


In [0]:
data_dep_eps, data_ind_eps = pate.perform_analysis(preds.numpy(), indices.numpy(), noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

AttributeError: ignored